<a href="https://colab.research.google.com/github/AleDan-Facile/Script_entita_da_condividere/blob/main/Da_Condividere_Script_Analisi_Entit%C3%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script per l'analisi delle entità nei testi - Facile.it

**IMPORTANTE: FAI UNA COPIA DEL FILE PER POTERLO USARE**

## Occorrente:
- File di autorizzazione per usare le API di Google, caricato in una cartella Drive di tua scelta.
- Un file CSV con due colonne: "URL" e "Content". "URL" serve per capire a quale URL, appunto, appartiene il contenuto. "Content" per il contenuto da analizzare. Puoi ottenere questo file a partire da una scansione di Screaming Frog con estrazione personalizzata, o con il metodo di scraping che preferisci. **Ricorda di caricare subito il file**.

### Impostazioni:

Nel primo blocco di codice troverai tre campi da impostare:

- lang: È il parametro che si riferisce alla lingua. Usa un input di codice linguistico a due caratteri. L'italiano è "it", ed è preimpostato.
- file: È il percorso temporaneo del CSV da analizzare. Ti basterà impostare: {{nome_file}} + ".csv"
- auth_path: È il percorso del file di autorizzazione JSON delle API di Google Cloud, che dovrai aver preventivamente salvato in Google Drive. Copia il percorso del file ed inseriscilo dove richiesto.

### Come far funzionare lo script:

Dopo aver impostato quanto richiesto, ti basta far partire il codice così come indicato per ottenere i CSV di risultato.
Otterrai diversi CSV, che dovrai combinare in un solo file per arrivare al risultato finale.

### Next Step:

Ci piacerebbe sapere cosa pensi che si possa migliorare di questo file, in modo da rendere il lavoro sempre più snello e veloce. Al momento, abbiamo in cantiere:
- Implementazione dello scraping via Trafilatura per essere indipendenti dal CSV, e passare direttamente un DataFrame alle API. Questo per snellire il processo e non uscire nemmeno dal Colab, anche in fase di preparazione dei dati.

In [ ]:
#@title Esegui questo blocco di codice per primo. Accetta la richiesta di accesso a Google Drive.

# Questo codice aggiorna alcune librerie che vengono richiamate nello script, e monta drive nell'attuale runtime.
# Montare drive serve a recuperare la cartella dove devi aver preventivamente inserito la chiave di autorizzazione JSON per utilizzare le API.
# Puoi aprire una chiave di autorizzazione qui: https://console.cloud.google.com/

!pip install --upgrade google-cloud-language
!pip install --upgrade requests

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Imposta i parametri richiesti (Descrizione dei campi come commento)

# lang è la lingua: devi inserire solo una sigla formata da due lettere. L'italiano "it" è inserito di default.
lang = 'it' #@param {type:"string"}

# file è il percorso del file CSV che devi caricare per l'analisi
file = 'input.csv' #@param {type:"string"}

# auth_path è il percorso del file di autorizzazione JSON del tuo Google Drive. Incollalo qui.
auth_path = '/xyz/xyz/xyz.json' #@param {type:"string"}

In [ ]:
#@title Esegui dopo aver impostato i parametri. Gli output saranno tra i files con nome "output.csv", "output_mentions.csv"; "output_salience.csv" e un file con la lista delle URL analizzate.
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=auth_path

#ensure the path is set correctly
!echo $GOOGLE_APPLICATION_CREDENTIALS

import requests
import json
import pandas as pd
import numpy as np
import csv
import re
import time

from google.cloud import language
from google.oauth2 import service_account
from google.cloud import language_v1

def analyze_entities(text_content):
    """
    Analyzing Entities in a String

    Args:
      text_content The text content to analyze
    """

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = lang
    document = {"content": text_content, "type_": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8

    response = client.analyze_entities(request = {'document': document, 'encoding_type': encoding_type})

    # Loop through entitites returned from the API

    aOutput = []
    aEntitiesTypeExclusion = ['PRICE','DATE','ADDRESS','NUMBER','PHONE_NUMBER']
    for entity in response.entities:
        
        if(language_v1.Entity.Type(entity.type_).name in aEntitiesTypeExclusion):
          continue

        salience = entity.salience
        metas = []
        mentions = []
        for metadata_name, metadata_value in entity.metadata.items():
            metas.append([metadata_name, metadata_value])

        # Loop over the mentions of this entity in the input document.
        # The API currently supports proper noun mentions.
        for mention in entity.mentions:
            #print(mention)
            mentions.append(mention.text.content+" - "+str(mention.text.begin_offset))

        #aOutput[0].append([entity.name,language_v1.Entity.Type(entity.type_).name])
        aOutput.append([entity.name,language_v1.Entity.Type(entity.type_).name,metas,mentions,entity.salience])

    return aOutput

def analyze_syntax(text_content):
  type_ = language_v1.Document.Type.PLAIN_TEXT
  language = lang
  document = {"content": text_content, "type_": type_, "language": language}

  # Available values: NONE, UTF8, UTF16, UTF32
  encoding_type = language_v1.EncodingType.UTF8
  aOutputSyntax = []
  response = client.analyze_syntax(request = {'document': document, 'encoding_type': encoding_type})
  for sentence in response.sentences:
    text = sentence.text
    aOutputSyntax.append([sentence.text.content,str(sentence.text.begin_offset)])
  return aOutputSyntax


client = language_v1.LanguageServiceClient()

df = pd.read_csv(file)
aOutput = dict()
aOutputSailence = dict()
aOutputMentions = dict()
aOutputSentences = dict()
entities = []
for label, row in df.iterrows():
    print("analyzing===>")
    print(row[0])
    a = []
    aSailence = dict()
    aMentions = dict()
    #aEntities = analyze_entities(row[2].lower())
    #aSentences = analyze_syntax(row[2].lower())
    #Qui viene analizzato il testo
    aEntities = analyze_entities(row[1])
    aSentences = analyze_syntax(row[1])
    for aEntity in aEntities:
      entityKey = aEntity[0]+" - "+aEntity[1]
      if(entityKey in a):
        continue;
      a.append(entityKey)
      aSailence[entityKey] = aEntity[4]
      aMentions[entityKey] = ";".join(aEntity[3])
    #print(aSailence)
    #Qui ricostruisce output
    aOutput[row[0]] = a
    aOutputSailence[row[0]] = aSailence
    aOutputMentions[row[0]] = aMentions
    aOutputSentences[row[0]] = aSentences
#concludere a stampa su csv dei doc di appoggio sailence e mentions
    
    #print(aOutputMentions)
    in_first = set(entities)
    in_second = set(a)
    in_second_but_not_in_first = in_second - in_first
    entities = entities + list(in_second_but_not_in_first)
    time.sleep(0.10)
#print(aOutputSailence)
result = {}
aTest = dict()
aSailence = dict()
aMentions = dict()

for key in aOutput.keys():
  aAux = dict()
  aAuxSailence = dict()
  aAuxMentions = dict()
  for val in entities:
    bInValues = 0
    if val in aOutput[key]:
      bInValues = 1
    aAux[val] = bInValues
    aAuxSailence[val] = aOutputSailence[key].get(val,0)
#    if val in aOutputSailence[key]:
#      aAuxSailence[val] = aOutputSailence[key][val]
    aAuxMentions[val] = aOutputMentions[key].get(val,"")
#    if val in aOutputMentions[key]:
#      aAuxMentions[val] = aOutputMentions[key][val]
  aTest[key] = aAux
  aSailence[key] = aAuxSailence
  aMentions[key] = aAuxMentions
dfAux = pd.DataFrame(aOutputSentences[key])
sSlug = re.sub('[^0-9a-zA-Z]+', '*', key)
sFileName = '/content/'+sSlug+'_sentences.csv'
dfAux.to_csv(sFileName)




df = pd.DataFrame(aTest)

df.to_csv('/content/output.csv')

df2 = pd.DataFrame(aSailence)
df2.to_csv('/content/output_sailence.csv')

df3 = pd.DataFrame(aMentions)
df3.to_csv('/content/output_mentions.csv')